# 1. Preparation

## 1.1 Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/Uni\ Leipzig/SS2022/.ssh ~/

## 1.2 Get correct CuDNN version
See https://github.com/googlecolab/colabtools/issues/2600



## 1.3 Install necessary packages

In [ ]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset


In [ ]:
!pip install python-terrier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.9 MB/s 
     |████████████████████████████████| 69 kB 9.8 MB/s 
     |████████████████████████████████| 303 kB 68.2 MB/s 
     |████████████████████████████████| 46 kB 5.1 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
     |████████████████████████████████| 294 kB 72.8 MB/s 
     |████████████████████████████████| 72 kB 1.8 MB/s 
     |████████████████████████████████| 126 kB 69.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 287 kB 67.5 MB/s 
     |████████████████████████████████| 59 kB 7.9 MB/s 
  Created wheel for python-terrie

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.6 MB/s 
     |████████████████████████████████| 141 kB 62.7 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 115 kB 69.4 MB/s 
     |████████████████████████████████| 212 kB 64.6 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import re
import datasets
import os
import pyterrier as pt
import pandas as pd
import numpy as np

In [ ]:

dataset_definitions = [
    #{"path": "glue", "name": "mrpc"},
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}
]
generate_csv = False
if generate_csv:
    for definition in dataset_definitions:
        dataset = datasets.load_dataset(path=definition["path"], name=definition["name"])
        for split in ["train", "validation", "test"]:
            out = []
            if split not in dataset:
                continue
            dataset_part = pd.DataFrame(dataset[split])
            #dataset_post = dataset_part.map(lambda x: para(model, tokenizer, x), num_proc=1)
            for idx, row in dataset_part.iterrows():
                out.append(para(row, split=split, path=definition["path"], name=definition["name"]))
            df = pd.DataFrame(out)
            df = df[df["original_label"] == 1]
            df = df.reset_index()
            df.to_csv(f'raw/out_{definition["path"]}_{definition["name"]}_{split}.csv', index=False)

In [ ]:

if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

In [ ]:
dataset_definitions = [
    #{"path": "glue", "name": "mrpc"},
    {"path": "paws", "name": "quora"},
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}
]
max_res = 50
total_num_sentences = 6
for definition in dataset_definitions:
    dname = definition["name"]
    dpath = definition["path"]
    for split in ["train", "validation", "test"]:
        if not os.path.isfile(f"raw/out_{dpath}_{dname}_{split}.csv"):
            continue
        anchors_with_para = pd.read_csv(f"raw/out_{dpath}_{dname}_{split}.csv").fillna("")

        if not os.path.isdir(f"./indices/pd_index_{dpath}_{dname}_{split}"):
            df = {"docno": [], "text": []}
            for index, row in anchors_with_para.iterrows():
                for sent in ["anchor", "para"]:
                    if sent and sent.strip() !="" and sent != "None":
                        df["text"].append(row[sent])
                        df["docno"].append(str(index) + "_" + sent)
            df = pd.DataFrame.from_dict(df)
            print(df)
            # index the text, record the docnos as metadata
            pd_indexer = pt.DFIndexer(f"./indices/pd_index_{dpath}_{dname}_{split}")
            indexref = pd_indexer.index(df["text"], df["docno"])
            batch_ret = pt.BatchRetrieve(indexref,num_results=max_res)
        else:
            #pd_indexer = pt.DFIndexer(f"./indices/pd_index_{dpath}_{dname}_{split}")
            batch_ret = pt.BatchRetrieve(f"./indices/pd_index_{dpath}_{dname}_{split}",num_results=max_res)

        
        if os.path.isfile(f"neg/out_{dpath}_{dname}_{split}_processed.csv"):
            continue
        
        out_list =  []
        for idx_row,row in anchors_with_para.iterrows():
            out_dict= {}
            for i in range(1, total_num_sentences+1):
                out_dict[f"sentence{i}"] = ""
            
            out_dict["sentence1"] = row["anchor"]
            out_dict["sentence2"] = row["para"]

            s = re.sub(r'[^a-zA-Z0-9\s]', ' ', row["anchor"])

            res = batch_ret(s)
            score_prev = -1
            is_too_similar = True
            sen_num = 3
            used_idx_nums = []
            for _ ,r in res.iterrows():
                if is_too_similar:
                    if score_prev != -1 and r['score']/score_prev < 0.7:
                        is_too_similar = False
                    else:
                        score_prev = r['score']
                        continue
                idx_num, sent = r["docno"].split("_", 1)[0],  r["docno"].split("_", 1)[1]

                # skip idx if it was already used, to make results more diverse

                if idx_num not in used_idx_nums:
                    used_idx_nums.append(idx_num)
                    out_dict[f"sentence{sen_num}"] = anchors_with_para.loc[int(idx_num)][sent]
                else:
                    continue
                
                # break if enough negatives are collected
                if sen_num == total_num_sentences:
                    break
                sen_num += 1

            all_sentences_valid = True
            for key, value in out_dict.items():
                if value == "":
                    all_sentences_valid = False
            
            if all_sentences_valid:
                out_list.append(out_dict)
                    

        out_df = pd.DataFrame(out_list)

        out_df.to_csv(f"neg/out_{dpath}_{dname}_{split}_processed.csv", index_label="idx")

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single que

,sentence1,sentence2,sentence3,sentence4,sentence5,sentence6
0,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,The 1979 -- 80 National Basketball Association...,The NBA season between 1979 and 80 was the 34t...,The season from 1982 to 83 National Basketball...,The NBA season between 1982 and 83 was the 37t...
1,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,"In 2011 , the literacy rate of West Bengal vil...","In 2011 , the literacy rate of West Bengal vil...","Note : The votes were cast on January 20 , but...","Note : the votes were cast on 20 January , but..."
2,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,Ernakulam is the seat of the North Paravur Add...,Ernakulam is the seat of North Paravur Additio...,"The district of Waitaki , in the regions of Ca...","The Waitaki district , in the Canterbury and O..."
3,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",William Henry Emerson was born in 1860 in Tunn...,William Henry Emerson was born in Tunnel Hill ...,"Caleb J. Emerson was born in Tunnel Hill , Geo...","Caleb J. Emerson was born in Tunnel Hill , Geo..."
4,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,The classical Lie - Algebras are finite - dime...,The classical Lie algebras are finite-dimensio...,It also hosts occasional BriSCA Formula 1 Stoc...,It also houses occasional BriSCA Formula 1 Sto...


In [ ]:
for key, val in out_df.items():
    print(len(val))

3539
3539
3539
3539
3539
3538
